In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import datetime as dt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

In [2]:
finalSeries = []

for i in range(2000,2017):
    filename = "../data/data"+str(i)
    for j in range(1,13):
        filename = filename + "month" + str(j) +".csv"
        data = pd.read_csv(filename, usecols=['NO2 Mean'])
        series = data['NO2 Mean'].tolist()
        if(len(series)<31 and len(series)!=0):
            while len(series)!=31:
                last = series[len(series)-1]
                series.append(last)
        if(len(series)!=0):
            series.append(j)
            seriesNP = np.asarray(series)
            finalSeries.append(seriesNP)
        filename = "../data/data"+str(i)

finalSeriesNP = np.asarray(finalSeries)

In [3]:
def DynamicTimeWarping(ts1, ts2, window):
    DTW_Dictionary={}
    
    window = max(window, abs(len(ts1)-len(ts2)))
    
    for i in range(-1,len(ts1)):
        for j in range(-1,len(ts2)):
            DTW_Dictionary[(i, j)] = float('inf')
    DTW_Dictionary[(-1, -1)] = 0
  
    for item in range(len(ts1)):
        for j in range(max(0, item-window), min(len(ts2), item+window)):
            distance= (ts1[item]-ts2[j])**2
            DTW_Dictionary[(item, j)] = distance + min(DTW_Dictionary[(item-1, j)],DTW_Dictionary[(item, j-1)], DTW_Dictionary[(item-1, j-1)])
        
    return np.sqrt(DTW_Dictionary[len(ts1)-1, len(ts2)-1])


def LowerBoundKeogh(ts1,ts2,reach):
    lowerBoundSum=0
    for index, item in enumerate(ts1):
        if len(ts2[(index-reach if index-reach>=0 else 0):(index+reach)])==0:
            break
        lowerBound = min(ts2[(index-reach if index-reach>=0 else 0):(index+reach)])
        upperBound = max(ts2[(index-reach if index-reach>=0 else 0):(index+reach)])
        
        if item>upperBound:
            lowerBoundSum = lowerBoundSum+(item-upperBound)**2
        elif item<lowerBound:
            lowerBoundSum = lowerBoundSum+(item-lowerBound)**2
    
    return np.sqrt(lowerBoundSum)

In [7]:
from sklearn.metrics import classification_report

def knn(train,test,w):
    preds=[]
    for ind,i in enumerate(test):
        min_dist=float('inf')
        closest_seq=[]
        #print ind
        for j in train:
            if LowerBoundKeogh(i[:-1],j[:-1],5)<min_dist:
                dist=DynamicTimeWarping(i[:-1],j[:-1],w)
                if dist<min_dist:
                    min_dist=dist
                    closest_seq=j
        preds.append(closest_seq[-1])
    return classification_report(test[:,-1],preds), preds

In [8]:
#np.random.shuffle(finalSeriesNP)
train, test = finalSeriesNP[:150,:], finalSeriesNP[150:,:]


In [9]:
report, pred = knn(train,test,10)

C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [10]:
print(pred)
print(test[:,-1])
print(report)

[8.0, 8.0, 6.0, 3.0, 1.0, 11.0, 1.0, 7.0, 9.0, 8.0, 8.0, 6.0, 6.0, 8.0, 8.0, 3.0, 3.0, 2.0, 9.0, 4.0, 7.0, 7.0, 8.0, 6.0, 7.0, 8.0, 8.0, 2.0, 3.0, 9.0, 5.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 5.0, 10.0, 2.0, 11.0, 4.0, 8.0]
[  7.   9.  10.  11.  12.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.
  11.  12.   1.   2.   3.   4.   5.   6.   8.   9.  10.  11.  12.   1.   2.
   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.   1.   2.   3.]
             precision    recall  f1-score   support

        1.0       0.00      0.00      0.00         4
        2.0       0.00      0.00      0.00         4
        3.0       0.00      0.00      0.00         4
        4.0       0.00      0.00      0.00         3
        5.0       0.00      0.00      0.00         3
        6.0       0.00      0.00      0.00         3
        7.0       0.10      0.33      0.15         3
        8.0       0.00      0.00      0.00         3
        9.0       0.00      0.00      0.00         4
       10.0       0.00      0.0